## Скачиваем отзывы 

In [1]:
import urllib.request
import re
import os
from bs4 import BeautifulSoup
import requests

In [11]:
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [31]:
from collections import Counter

### Берем все отзывы со страницы. Каждый пользователь оценивает магазин и доставку, поэтому нам необходимо выкидывать из выборки каждый второй отзыв. 

In [2]:
#функция делает список ссылок на страницы с отзывами
def list_of_urls (url):
    list_of_urls = []
    for i in range (2, 8):
        new_url = url + '&page=' + str(i)
        list_of_urls.append(new_url)
    return list_of_urls

In [3]:
#функция пробегается по страницам с отзывами, собирает отзывы, выкидывая из них каждый второй
def comments (list_of_urls, file):
    for url in list_of_urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text)
        comments = soup.find_all('p', {'class': 'review-text'})
        for i in range (0, len(comments), 2):
            txt = str (comments[i])
            txt1 = re.sub ('<.*>','', txt)
            text = re.sub ('\s+', ' ', txt1) 
            with open(file, 'a', encoding='utf-8') as f:
                f.write (text + '\n') 

In [4]:
with open('negative_comments.txt', 'w', encoding='utf-8') as f:
    f.write('')

In [5]:
with open('positive_comments.txt', 'w', encoding='utf-8') as f:
    f.write('')

In [6]:
#ссылки на страницы с негативными отзывами
urls_neg = list_of_urls ('https://shop.tastycoffee.ru/reviews?shop=1')

In [7]:
#ссылки на страницы с положительными отзывами
urls_pos = list_of_urls ('https://shop.tastycoffee.ru/reviews?shop=5')

In [8]:
#выкачиваем негативные отзывы (58)
comments(urls_neg, 'negative_comments.txt')

In [9]:
#выкачиваем положительные отзывы (60)
comments(urls_pos, 'positive_comments.txt')

In [44]:
#отзывы для проверки
def comments_chek (file):
    with open(file, 'r', encoding='utf-8') as f:
        lines = f.readlines()[54:]
    return lines

In [45]:
comments_10 = []
comments_10.extend(comments_chek ('negative_comments.txt'))
comments_10.extend(comments_chek ('positive_comments.txt'))

### Токенизируем и приведем к начальной форме

In [27]:
#функция токенизирует и приводит слова в отзывах к начальной форме
def token_lemma (file):
    words = []
    tokens = []
    with open(file, 'r', encoding='utf-8') as f:
        lines = f.readlines()[0:55]
        for line in lines:
            tokens.extend([w.lower() for w in word_tokenize(line) if w.isalpha()])
            for token in tokens:
                ana = morph.parse(token)[0].normal_form
                words.append(ana)
    return words

In [28]:
words_neg = token_lemma ('negative_comments.txt')

In [30]:
words_pos = token_lemma ('positive_comments.txt')

In [32]:
freq_pos = Counter(words_pos)
freq_neg = Counter(words_neg)

In [40]:
#создадим частотные словари со словами, которые встречаются только в негативных/позитивных отзывах
only_neg = {}
only_pos = {}
for key in freq_pos:
    if key not in freq_neg and freq_pos[key] > 5:
        only_pos[key] = freq_pos[key]
        
for key in freq_neg:
    if key not in freq_pos and freq_neg[key] > 5:
        only_neg[key] = freq_neg[key]

### Функция, которая определяет положительный отзыв или отрицательный

In [59]:
def comment_detect(only_pos, only_neg, text): 
    counts = {'pos' : 0, 'neg' : 0}
    words = []
    pos = 0
    neg = 0
    for token in nltk.word_tokenize(text):
        ana = morph.parse(token)[0].normal_form
        words.append(ana) 
    for word in words:
        if word in only_pos:
            counts['pos'] += 1
        elif word in only_neg:
            counts['neg'] += 1
        else:
            continue
    if counts['pos'] > counts['neg']:
        mood = 'positive'
    elif counts['pos'] == counts['neg']:
        mood = 'neutral'
    else:
        mood = 'negative'
    return mood

В нашем списке отзывов для проверки первые 4 отзыва отрицательные, остальные 6 - положительные.

In [56]:
!pip install sklearn

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [57]:
from sklearn.metrics import accuracy_score

In [60]:
#оценка качества при помощи accuracy
def accuracy (comments_10, only_pos, only_neg):
    results = []
    gold = []
    for comment in comments_10[0:4]:
        results.append(comment_detect(only_pos, only_neg, comment))
        gold.append('negative')
    for comment in comments_10[4:10]:
        results.append(comment_detect(only_pos, only_neg, comment))
        gold.append('positive')
    print("Accuracy: %.4f" % accuracy_score(results, gold))        

In [61]:
accuracy(comments_10, only_pos, only_neg)

Accuracy: 0.8000


### Предложения по улучшению программы

Во-первых, для точности программы было бы неплохо накачать большее количество отзывов. Во-вторых, в ходе выполнения программы, я заметила, что она достаточно туго справлялась с токенизацией и приведением к начальной форме слов из отзывов. Изначально для этого я попыталась использовать mystem, так как качество его лемматизации лучше, но скорость программы в данном случае сильно страдала. В связи с этим, было принято решение использовать nltk и pymorthy, что значительно ускорило программу. Однако стоит отметить, что файлы с отзывами я прочитывала через readlines(), что не подойдет, если файлы будут весить гораздо больше. Большие файлы следует читать построчно. 